In [11]:
import pandas as pd
import re
import numpy as np

formula = pd.read_excel('sample.xlsx', sheet_name='formula')
infor = pd.read_excel('sample.xlsx', sheet_name='infor')    

In [12]:
def split_general_formula(row):
    try:
        formula_str = str(row['formula']).strip()
        match = re.match(r"([A-Za-z0-9\-]+(?:-[A-Za-z0-9\-]+)*)\s*\(([\d\.\-]+(?:-[\d\.\-]+)*)\s*mol\)", formula_str)
        if not match:
            return pd.Series()
        compounds_part, mols_part = match.groups()
        compounds = compounds_part.split("-")
        mols = [float(m) for m in mols_part.split("-")]
        if len(compounds) != len(mols):
            return pd.Series()
        data = {}
        for i, (compound, mol_val) in enumerate(zip(compounds, mols), start=1):
            data[f"c{i}"] = compound
            data[f"n{i}"] = mol_val
        return pd.Series(data)
    except:
        return pd.Series()

df_result = formula.apply(split_general_formula, axis=1)
df_result = df_result.dropna(how='all')
formula = pd.concat([formula, df_result], axis=1)

In [13]:
o_dict = dict(zip(infor['compound'], infor['O']))
f_dict = dict(zip(infor['compound'], infor['F']))
c_dict = dict(zip(infor['compound'], infor['C']))
h_dict = dict(zip(infor['compound'], infor['H']))

for i in formula.index:
    s_count = 1  
    a_count = 1
    Li = 0
    sol = 0
    
    for j in range(1, 9):
        c_col = f'c{j}'
        n_col = f'n{j}'
        if c_col in formula.columns and pd.notna(formula.loc[i, c_col]):
            compound = formula.loc[i, c_col]
            n_value = formula.loc[i, n_col]
            
            if 'Li' in compound:
                Li += n_value
                for k in o_dict.keys():
                    if compound in k:
                        formula.loc[i, f'a{a_count}O'] = o_dict[k] if pd.notna(o_dict[k]) else 0
                        formula.loc[i, f'a{a_count}F'] = f_dict[k] if pd.notna(f_dict[k]) else 0
                        a_count += 1
                        break
            else:
                sol += n_value
                for k in o_dict.keys():
                    if compound in k:
                        formula.loc[i, f's{s_count}C'] = c_dict[k] if pd.notna(c_dict[k]) else 0
                        formula.loc[i, f's{s_count}H'] = h_dict[k] if pd.notna(h_dict[k]) else 0
                        formula.loc[i, f's{s_count}O'] = o_dict[k] if pd.notna(o_dict[k]) else 0
                        formula.loc[i, f's{s_count}F'] = f_dict[k] if pd.notna(f_dict[k]) else 0
                        s_count += 1
                        break
    
    formula.loc[i, 'Li'] = Li
    formula.loc[i, 'sol'] = sol
    formula.loc[i, 'Li/sol'] = round(Li/sol, 3) if sol != 0 else 0


In [14]:
prefix_suffix_pairs = [
    ('a', 'O'),
    ('a', 'F'),
    ('s', 'C'),
    ('s', 'H'),
    ('s', 'O'),
    ('s', 'F')
]

for prefix, suffix in prefix_suffix_pairs:
    cols = [col for col in formula.columns if col.startswith(prefix) and col.endswith(suffix) and len(col) == 3]
    formula[prefix + suffix] = formula[cols].sum(axis=1, skipna=True)

formula['sF/sC'] = formula.apply(lambda x: round(x['sF'] / x['sC'], 2) if pd.notna(x['sF']) and pd.notna(x['sC']) and x['sC'] != 0 else 0, axis=1)
formula['sF/sO'] = formula.apply(lambda x: round(x['sF'] / x['sO'], 2) if pd.notna(x['sF']) and pd.notna(x['sO']) and x['sO'] != 0 else 0, axis=1)
formula['sC/sH'] = formula.apply(lambda x: round(x['sC'] / x['sH'], 2) if pd.notna(x['sC']) and pd.notna(x['sH']) and x['sH'] != 0 else 0, axis=1)
formula['sO/sC'] = formula.apply(lambda x: round(x['sO'] / x['sC'], 2) if pd.notna(x['sO']) and pd.notna(x['sC']) and x['sC'] != 0 else 0, axis=1)

In [15]:
smiles_dict = dict(zip(infor['compound'], infor['SMILES']))
formula['dbO'] = 0
for i in formula.index:
    db_count = 0
    for j in range(1, 9):
        c_col = f'c{j}'
        if c_col in formula.columns and pd.notna(formula.loc[i, c_col]):
            compound = formula.loc[i, c_col]
            for k in smiles_dict.keys():
                if compound in k and pd.notna(smiles_dict[k]):
                    db_count += smiles_dict[k].count('=')
                    break
    formula.loc[i, 'dbO'] = db_count
formula['sbO'] = formula['sO'] - formula['dbO']

In [ ]:
formula

In [16]:
with pd.ExcelWriter('result.xlsx') as writer:
    formula.to_excel(writer, sheet_name='formula', index=True)
    infor.to_excel(writer, sheet_name='infor', index=True)